/* QUESTIONS 
/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */

QUERY:
SELECT name FROM Facilities
WHERE EXISTS
(SELECT membercost FROM Facilities WHERE membercost!=0);



/* Q2: How many facilities do not charge a fee to members?
 */9 
 4 Facilities do not charge fee to members: 
QUERY:
SELECT COUNT(*) FROM Facilities
WHERE membercost=0;



/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */
QUERY:
SELECT facid,name,membercost,monthlymaintenance FROM Facilities
WHERE membercost!=0 AND membercost<0.20*monthlymaintenance;


/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */
QUERY:
SELECT * FROM Facilities
WHERE facid IN (1,5);


/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */

QUERY:
SELECT name, monthlymaintenance, 
CASE WHEN monthlymaintenance > 100 THEN 'expensive'
     ELSE 'cheap' END AS label
FROM Facilities

/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */

QUERY:
SELECT firstname,surname
FROM Members
WHERE joindate=(SELECT MAX(joindate) FROM Members);

/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */

QUERY:
select distinct mems.firstname || ' ' || mems.surname as member, facs.name as facility
	from 
		cd.members mems
		inner join cd.bookings bks
			on mems.memid = bks.memid
		inner join cd.facilities facs
			on bks.facid = facs.facid
	where
		bks.facid in (0,1)
order by member;          


/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */


QUERY:
SELECT CONCAT(surname, ', ', firstname) AS Full_Name, name AS location, CASE WHEN firstname = 'GUEST' 
THEN guestcost slots ELSE membercost slots END AS cost 
FROM Bookings AS b JOIN Members AS m ON m.memid = b.memid JOIN Facilities AS f ON f.facid = b.facid 
WHERE starttime >= '2019-09-14' AND starttime < '2019-09-15' AND CASE WHEN firstname = 'GUEST' THEN guestcost slots 
ELSE membercost slots END > 30 ORDER BY cost DESC';

/* Q9: This time, produce the same result as in Q8, but using a subquery. */


QUERY:
SELECT CONCAT(surname, ', ', firstname) AS Full_Name, f.name AS location, cost, 
FROM (Select firstname, surname, name, starttime, CASE WHEN firstname = 'GUEST' THEN guestcost slots ELSE membercost slots END AS cost, 
FROM members INNER JOIN bookings ON members.memid = bookings.memid INNER JOIN Facilities ON bookings.facid = facilities.facid) AS inner_table 
WHERE starttime >= '2019-09-14' AND starttime < '2019-09-15' AND cost > 30 ORDER BY cost DESC;

In [2]:
import sqlite3
from sqlite3 import Error

In [3]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! 

In [4]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT name, SUM(CASE WHEN memid = 0 THEN guestcost * slots ELSE membercost * slots
        END) AS revenue
        FROM Facilities
        INNER JOIN Bookings ON Facilities.facid = Bookings.facid
        GROUP BY name
        HAVING revenue < 1000
        ORDER BY revenue;
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Table Tennis', 180)
('Snooker Table', 240)
('Pool Table', 270)


In [5]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT m.firstname || ' ' || m.surname || ' recommended by' || ' ' || Members.firstname || ' ' || Members.surname
        FROM Members AS m
        INNER JOIN Members 
        ON m.recommendedby = Members.memid
        WHERE Members.memid != 0
        ORDER BY m.surname, m.firstname;
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Florence Bader recommended by Ponder Stibbons',)
('Anne Baker recommended by Ponder Stibbons',)
('Timothy Baker recommended by Jemima Farrell',)
('Tim Boothe recommended by Tim Rownam',)
('Gerald Butters recommended by Darren Smith',)
('Joan Coplin recommended by Timothy Baker',)
('Erica Crumpet recommended by Tracy Smith',)
('Nancy Dare recommended by Janice Joplette',)
('Matthew Genting recommended by Gerald Butters',)
('John Hunt recommended by Millicent Purview',)
('David Jones recommended by Janice Joplette',)
('Douglas Jones recommended by David Jones',)
('Janice Joplette recommended by Darren Smith',)
('Anna Mackenzie recommended by Darren Smith',)
('Charles Owen recommended by Darren Smith',)
('David Pinker recommended by Jemima Farrell',)
('Millicent Purview recommended by Tracy Smith',)
('Henrietta Rumney recommended by Matthew Genting',)
('Ramnaresh Sarwin recommended by Florence Bader',)
('Jack Smith recommended by Darren Smith',)
('Ponder Stibbon

In [6]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT f.name AS facility, f_mem_count.books
        FROM Facilities AS f
        INNER JOIN (SELECT facid, COUNT(*) AS books
        FROM Bookings AS b
        WHERE memid != 0
        GROUP BY facid) AS f_mem_count ON f.facid = f_mem_count.facid
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Tennis Court 1', 308)
('Tennis Court 2', 276)
('Badminton Court', 344)
('Table Tennis', 385)
('Massage Room 1', 421)
('Massage Room 2', 27)
('Squash Court', 195)
('Snooker Table', 421)
('Pool Table', 783)


In [7]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT f.name AS facility, f_mem_count.month, f_mem_count.books
        FROM Facilities AS f
        INNER JOIN (SELECT facid, strftime('%Y-%m', starttime) AS month, COUNT(*) AS books
        FROM Bookings AS b
        WHERE memid != 0
        GROUP BY facid, month) AS f_mem_count ON f.facid = f_mem_count.facid
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Tennis Court 1', '2012-07', 65)
('Tennis Court 1', '2012-08', 111)
('Tennis Court 1', '2012-09', 132)
('Tennis Court 2', '2012-07', 41)
('Tennis Court 2', '2012-08', 109)
('Tennis Court 2', '2012-09', 126)
('Badminton Court', '2012-07', 51)
('Badminton Court', '2012-08', 132)
('Badminton Court', '2012-09', 161)
('Table Tennis', '2012-07', 48)
('Table Tennis', '2012-08', 143)
('Table Tennis', '2012-09', 194)
('Massage Room 1', '2012-07', 77)
('Massage Room 1', '2012-08', 153)
('Massage Room 1', '2012-09', 191)
('Massage Room 2', '2012-07', 4)
('Massage Room 2', '2012-08', 9)
('Massage Room 2', '2012-09', 14)
('Squash Court', '2012-07', 23)
('Squash Court', '2012-08', 85)
('Squash Court', '2012-09', 87)
('Snooker Table', '2012-07', 68)
('Snooker Table', '2012-08', 154)
('Snooker Table', '2012-09', 199)
('Pool Table', '2012-07', 103)
('Pool Table', '2012-08', 272)
('Pool Table', '2012-09', 408)
